In [65]:
# import thư viện
import pandas as pd
import numpy as np
from underthesea import word_tokenize, pos_tag, sent_tokenize
import regex
import string

In [66]:
#đọc dữ liệu
df1 =pd.read_csv('Danh_gia.csv',index_col=0)


In [67]:
#xem thông tin
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21575 entries, 1 to 21575
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ma_khach_hang       21575 non-null  int64 
 1   noi_dung_binh_luan  20674 non-null  object
 2   ngay_binh_luan      21575 non-null  object
 3   gio_binh_luan       21575 non-null  object
 4   so_sao              21575 non-null  int64 
 5   ma_san_pham         21575 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 1.2+ MB


In [68]:
# xem vài dòng
df1.head(15)

,ma_khach_hang,noi_dung_binh_luan,ngay_binh_luan,gio_binh_luan,so_sao,ma_san_pham
id,,,,,,
1,443,"SỬ DỤNG DỄ DÀNG, RẤT THOẢI MÁI, THƯ GIÃN TỘT ĐỘ.",29/04/2023,17:06,5,308500015
2,1030,"Sử dụng dễ dãng,rất thoải mái,thư giãn tột độ",30/04/2023,15:04,5,308500015
3,689,Mình rất thích hasaki va sp tẩy trang này,30/04/2023,18:34,5,422216594
4,2519,Sản phẩm có khả năng làm sạch tốt. Lớp trang đ...,17/07/2022,13:48,5,204100075
5,402,"Sữa rửa mặt tốt,sạch mụn,mịn da,đáng mua nha",15/04/2023,23:04,5,422208977
6,3014,NaN,3/1/2020,18:31,5,345800013
7,2517,"Sau 77 49 dòng srm, thì chân ái là đây, mua tu...",25/12/2022,17:11,5,204100019
8,1750,Đó giờ mình sài bha của obagi cũng 3-4 chai rồ...,9/7/2022,4:34,5,100220035
9,3394,Rất ok mình xài 2 chai rồi,7/7/2022,11:18,5,204100024


# Vì đây là dữ liệu text tiếng Việt nên cần thực hiện công việc chuẩn hóa text là chính  
<a> Chuẩn hoá cột noi_dung_binh_luan


In [69]:
##LOAD EMOJICON
file = open('files/emojicon.txt', 'r', encoding="utf8")
emoji_lst = file.read().split('\n')
emoji_dict = {}
for line in emoji_lst:
    key, value = line.split('\t')
    emoji_dict[key] = str(value)
#print(teen_dict)
file.close()
#################
#LOAD TEENCODE
file = open('files/teencode.txt', 'r', encoding="utf8")
teen_lst = file.read().split('\n')
teen_dict = {}
for line in teen_lst:
    key, value = line.split('\t')
    teen_dict[key] = str(value)
#print(teen_dict)
file.close()
###############
#LOAD TRANSLATE ENGLISH -> VNMESE
file = open('files/english-vnmese.txt', 'r', encoding="utf8")
englist_lst = file.read().split('\n')
for line in englist_lst:
    key, value = line.split('\t')
    teen_dict[key] = str(value)
#print(teen_dict)
file.close()
################
#LOAD wrong words
file = open('files/wrong-word.txt', 'r', encoding="utf8")
wrong_lst = file.read().split('\n')
file.close()
#################
#LOAD STOPWORDS
file = open('files/vietnamese-stopwords.txt', 'r', encoding="utf8")
stopwords_lst = file.read().split('\n')
file.close()

In [70]:
for item in df1['noi_dung_binh_luan'].head(10):
    print("******")
    print(item)

******
SỬ DỤNG DỄ DÀNG, RẤT THOẢI MÁI, THƯ GIÃN TỘT ĐỘ.
******
Sử dụng dễ dãng,rất thoải mái,thư giãn tột độ
******
Mình rất thích hasaki va sp tẩy trang này
******
Sản phẩm có khả năng làm sạch tốt. Lớp trang điểm trên da, mắt và môi nhanh chóng bị loại bỏ. Chỉ cần lau qua bằng một miếng bông tẩy trang là toàn bộ bụi bẩn, dầu thừa và lớp make-up đã được lấy đi. Vì thế, dùng cực kỳ tiết kiệm. Thêm nữa, em này cũng phù hợp cho da đang bị kích ứng. Bởi sản phẩm không chỉ làm sạch da mà còn làm dịu, giúp giảm tình trạng ngứa ngáy, khó chịu.
******
Sữa rửa mặt tốt,sạch mụn,mịn da,đáng mua nha
******
nan
******
Sau 77 49 dòng srm, thì chân ái là đây, mua tuyp nhỏ dùng thử trc, thấy ok nên nay mua chai 400ml luôn, nay là chai thứ 2 r ms lên đây đánh giá. Ns chung ai da nhờn, khô, hay hở tí là nổi mụn thì chai này ok nha, 100 điểm
******
Đó giờ mình sài bha của obagi cũng 3-4 chai rồi. Nay đổi qua thử sài của plc xem thế nào . Hasaki chuẩn bị giao hàng nhanh , có kèm quà tặng, hết quà thì có 

In [71]:
def process_text(text, dict_emoji, dict_teen, lst_wrong):
    document = text.lower()
    document = document.replace("’",'')
    document = regex.sub(r'\.+', ".", document)
    new_sentence =''
    for sentence in sent_tokenize(document):
        # if not(sentence.isascii()):
        ###### CONVERT EMOJICON
        sentence = ''.join(dict_emoji[word]+' ' if word in dict_emoji else word for word in list(sentence))
        ###### CONVERT TEENCODE
        sentence = ' '.join(dict_teen[word] if word in dict_teen else word for word in sentence.split())
        ###### DEL Punctuation & Numbers
        pattern = r'(?i)\b[a-záàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ]+\b'
        sentence = ' '.join(regex.findall(pattern,sentence))
        ###### DEL wrong words   
        sentence = ' '.join('' if word in lst_wrong else word for word in sentence.split())
        new_sentence = new_sentence+ sentence + '. '                    
    document = new_sentence  
    #print(document)
     ###### DEL excess blank space
    document = regex.sub(r'\s+', ' ', document).strip()
    return document

In [72]:
df1['content_new'] = df1['noi_dung_binh_luan'].apply(lambda x: process_text(str(x), emoji_dict, teen_dict, wrong_lst))

In [73]:
for item in df1['content_new'].head(10):
    print("******")
    print(item)

******
sử dụng dễ dàng rất thoải mái thư giãn tột độ.
******
sử dụng dễ dãng rất thoải mái thư giãn tột độ.
******
mình rất thích hasaki tẩy này.
******
sản phẩm có khả năng làm sạch tốt. lớp điểm trên mắt và môi chóng chóng bị loại bỏ. chỉ cần bằng một miếng bông tẩy là toàn bộ bụi bẩn dầu thừa và lớp đã được lấy đi. vì thế dùng cực kỳ tiết kiệm. thêm nữa này cũng phù hợp cho đang bị kích ứng. bởi sản phẩm không chỉ làm sạch mà còn làm dịu giúp giảm tình trạng ngứa ngáy khó chịu.
******
sữa rửa mặt tốt sạch mụn mịn đáng .
******
.
******
dòng srm thì chân ái là đây tuyp nhỏ dùng thử thấy đồng ý nên cái ghế luôn là cái ghế thứ lên đây đánh giá. nói nhờn khô cỏ khô hở tí là nổi mụn thì cái ghế này đồng ý điểm.
******
đó giờ mình sài bha của obagi cũng cái ghế rồi. đổi thử sài của plc thế nào. hasaki chuẩn bị hàng chóng có kèm quà tặng hết quà thì có trừ tiền cho mình.
******
rất đồng ý mình xài cái ghế rồi.
******
mình bị ứng không hợp sản phẩm được vượt cho bạn nào cần .


# Chuẩn hóa unicode tiếng việt

In [74]:
def loaddicchar():
    uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
    unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
 
# Đưa toàn bộ dữ liệu qua hàm này để chuẩn hóa lại
def covert_unicode(txt):
    dicchar = loaddicchar()
    return regex.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

In [75]:
df1['content_new'] = df1['content_new'].apply(lambda x: covert_unicode(str(x)))

In [76]:
for item in df1['content_new'].head(10):
    print("******")
    print(item)

******
sử dụng dễ dàng rất thoải mái thư giãn tột độ.
******
sử dụng dễ dãng rất thoải mái thư giãn tột độ.
******
mình rất thích hasaki tẩy này.
******
sản phẩm có khả năng làm sạch tốt. lớp điểm trên mắt và môi chóng chóng bị loại bỏ. chỉ cần bằng một miếng bông tẩy là toàn bộ bụi bẩn dầu thừa và lớp đã được lấy đi. vì thế dùng cực kỳ tiết kiệm. thêm nữa này cũng phù hợp cho đang bị kích ứng. bởi sản phẩm không chỉ làm sạch mà còn làm dịu giúp giảm tình trạng ngứa ngáy khó chịu.
******
sữa rửa mặt tốt sạch mụn mịn đáng .
******
.
******
dòng srm thì chân ái là đây tuyp nhỏ dùng thử thấy đồng ý nên cái ghế luôn là cái ghế thứ lên đây đánh giá. nói nhờn khô cỏ khô hở tí là nổi mụn thì cái ghế này đồng ý điểm.
******
đó giờ mình sài bha của obagi cũng cái ghế rồi. đổi thử sài của plc thế nào. hasaki chuẩn bị hàng chóng có kèm quà tặng hết quà thì có trừ tiền cho mình.
******
rất đồng ý mình xài cái ghế rồi.
******
mình bị ứng không hợp sản phẩm được vượt cho bạn nào cần .


# Xử lý postag_thesea

In [77]:
def process_special_word(text):
    new_text = ''
    text_lst = text.split()
    i= 0
    if 'không' in text_lst:
        while i <= len(text_lst) - 1:
            word = text_lst[i]
            #print(word)
            #print(i)
            if  word == 'không':
                next_idx = i+1
                if next_idx <= len(text_lst) -1:
                    word = word +'_'+ text_lst[next_idx]
                i= next_idx + 1
            else:
                i = i+1
            new_text = new_text + word + ' '
    else:
        new_text = text
    return new_text.strip()

In [78]:
def process_postag_thesea(text):
    new_document = ''
    for sentence in sent_tokenize(text):
        sentence = sentence.replace('.','')
        ###### POS tag
        lst_word_type = ['A','AB','V','VB','VY','R']
        sentence = ' '.join( word[0] if word[1].upper() in lst_word_type else '' for word in pos_tag(process_special_word(word_tokenize(sentence, format="text"))))
        new_document = new_document + sentence + ' '
    ###### DEL excess blank space
    new_document = regex.sub(r'\s+', ' ', new_document).strip()
    return new_document

In [79]:
df1['content_new'] = df1['content_new'].apply(lambda x: process_postag_thesea(str(x)))

In [80]:
for item in df1['content_new'].head(10):
    print("******")
    print(item)

******
dễ_dàng rất thoải_mái
******
dễ dãng rất thoải_mái
******
rất thích hasaki_tẩy
******
có làm sạch tốt bị loại_bỏ chỉ cần là bẩn dầu_thừa đã được lấy đi dùng thêm nữa cũng phù_hợp đang bị kích_ứng làm sạch mà_còn làm dịu giúp giảm tình_trạng
******
rửa tốt sạch mịn đáng
******

******
chân_ái là nhỏ dùng thử thấy luôn là lên nói khô khô là nổi
******
sài cũng đổi chuẩn_bị có kèm tặng hết có trừ
******
rất đồng_ý xài
******
bị ứng không_hợp sản_phẩm được vượt cần


# Remove Stopword

In [81]:
def remove_stopword(text, stopwords):
    ###### REMOVE stop words
    document = ' '.join('' if word in stopwords else word for word in text.split())
    #print(document)
    ###### DEL excess blank space
    document = regex.sub(r'\s+', ' ', document).strip()
    return document

In [82]:
df1['content_new'] = df1['content_new'].apply(lambda x: remove_stopword(str(x), stopwords_lst))

In [83]:
for item in df1['content_new'].head(10):
    print("******")
    print(item)

******
dễ_dàng thoải_mái
******
dãng thoải_mái
******
thích hasaki_tẩy
******
sạch tốt loại_bỏ bẩn dầu_thừa kích_ứng sạch mà_còn dịu giúp
******
rửa tốt sạch mịn
******

******
chân_ái thử khô khô nổi
******
sài đổi kèm tặng trừ
******
đồng_ý xài
******
ứng không_hợp sản_phẩm


In [84]:
# Loại bỏ dòng dữ liệu bị thiếu trong cột "noi_dung_binh_luan"
df1 = df1.dropna(subset=['noi_dung_binh_luan'])

In [85]:

# Hiển thị vài dòng dữ liệu đã được chuẩn hóa
df1.head(10)


,ma_khach_hang,noi_dung_binh_luan,ngay_binh_luan,gio_binh_luan,so_sao,ma_san_pham,content_new
id,,,,,,,
1,443,"SỬ DỤNG DỄ DÀNG, RẤT THOẢI MÁI, THƯ GIÃN TỘT ĐỘ.",29/04/2023,17:06,5,308500015,dễ_dàng thoải_mái
2,1030,"Sử dụng dễ dãng,rất thoải mái,thư giãn tột độ",30/04/2023,15:04,5,308500015,dãng thoải_mái
3,689,Mình rất thích hasaki va sp tẩy trang này,30/04/2023,18:34,5,422216594,thích hasaki_tẩy
4,2519,Sản phẩm có khả năng làm sạch tốt. Lớp trang đ...,17/07/2022,13:48,5,204100075,sạch tốt loại_bỏ bẩn dầu_thừa kích_ứng sạch mà...
5,402,"Sữa rửa mặt tốt,sạch mụn,mịn da,đáng mua nha",15/04/2023,23:04,5,422208977,rửa tốt sạch mịn
7,2517,"Sau 77 49 dòng srm, thì chân ái là đây, mua tu...",25/12/2022,17:11,5,204100019,chân_ái thử khô khô nổi
8,1750,Đó giờ mình sài bha của obagi cũng 3-4 chai rồ...,9/7/2022,4:34,5,100220035,sài đổi kèm tặng trừ
9,3394,Rất ok mình xài 2 chai rồi,7/7/2022,11:18,5,204100024,đồng_ý xài
10,1817,"Mik bị kich ứng, không hợp sản phẩm, sd được 3...",5/2/2024,3:50,4,422216354,ứng không_hợp sản_phẩm


In [86]:
import re
def fix_text_errors(text):
    common_errors = {
        "dãng": "dễ_dàng",
        "sài": "xài",
    }
    for wrong, correct in common_errors.items():
        text = re.sub(rf"\b{wrong}\b", correct, text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [87]:
df1['content_new'] = df1['content_new'].apply(fix_text_errors)


In [88]:
df1.to_csv('Danh_gia_cleaned_updated.csv', index=False)


In [ ]:
# xem vài dòng 
df1.head(10)

,ma_khach_hang,noi_dung_binh_luan,ngay_binh_luan,gio_binh_luan,so_sao,ma_san_pham,content_new
id,,,,,,,
1,443,"SỬ DỤNG DỄ DÀNG, RẤT THOẢI MÁI, THƯ GIÃN TỘT ĐỘ.",29/04/2023,17:06,5,308500015,dễ_dàng thoải_mái
2,1030,"Sử dụng dễ dãng,rất thoải mái,thư giãn tột độ",30/04/2023,15:04,5,308500015,dễ_dàng thoải_mái
3,689,Mình rất thích hasaki va sp tẩy trang này,30/04/2023,18:34,5,422216594,thích hasaki_tẩy
4,2519,Sản phẩm có khả năng làm sạch tốt. Lớp trang đ...,17/07/2022,13:48,5,204100075,sạch tốt loại_bỏ bẩn dầu_thừa kích_ứng sạch mà...
5,402,"Sữa rửa mặt tốt,sạch mụn,mịn da,đáng mua nha",15/04/2023,23:04,5,422208977,rửa tốt sạch mịn
7,2517,"Sau 77 49 dòng srm, thì chân ái là đây, mua tu...",25/12/2022,17:11,5,204100019,chân_ái thử khô khô nổi
8,1750,Đó giờ mình sài bha của obagi cũng 3-4 chai rồ...,9/7/2022,4:34,5,100220035,xài đổi kèm tặng trừ
9,3394,Rất ok mình xài 2 chai rồi,7/7/2022,11:18,5,204100024,đồng_ý xài
10,1817,"Mik bị kich ứng, không hợp sản phẩm, sd được 3...",5/2/2024,3:50,4,422216354,ứng không_hợp sản_phẩm
